<a href="https://colab.research.google.com/github/Danish-me/Implementation-RAG-with-Pinecone/blob/main/RAG_implementation_with_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [3]:
import time

index_name = "langchain-rag"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [5]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [6]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="Responsibilities: Develops, implements, and maintains the QMS. Ensures compliance with relevant quality standards and regulations. Leads quality audits and continuous improvement initiatives.",
    metadata={"source": "website"},
)

document_2 = Document(
    page_content="Quality Assurance (QA) Specialist: Responsibilities: Conducts audits, reviews processes, and ensures that quality standards are consistently met. Identifies and documents non-conformances and follows up on corrective actions.",
    metadata={"source": "website"},
)

document_3 = Document(
    page_content="Document Control Specialist: Responsibilities: Manages and controls all QMS documentation, ensuring documents are updated, reviewed, and distributed as needed. Maintains records of quality audits and reports.",
    metadata={"source": "website"},
)

document_4 = Document(
    page_content="Compliance Officer: esponsibilities: Ensures that the organization complies with external regulations and standards related to quality. Prepares for and manages external audits.",
    metadata={"source": "website"},
)

document_5 = Document(
    page_content="Training Coordinator: Responsibilities: Develops and delivers training programs related to quality standards and procedures. Ensures that all employees understand their role in the QMS.",
    metadata={"source": "website"},
)

document_6 = Document(
    page_content="Process Engineer: Responsibilities: Designs and optimizes production processes to ensure efficiency and quality. Works closely with QA and QC teams to implement improvements.",
    metadata={"source": "website"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['bcdfbece-27d3-4cf4-8517-7dad6bc6beef',
 'e95bccc6-3b68-4cdb-8d2e-b998bcc0eabe',
 'f3edd774-6927-4743-a9de-839bd639d2ab',
 'ac4778ae-930a-46df-9101-f94d0fb85188',
 '67399b23-81be-4769-87ed-e2cb43d4c5f6',
 'f02683c3-d7b8-436c-b0b3-6c4218f1b7f7']

In [7]:
from uuid import uuid4

from langchain_core.documents import Document

In [8]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['e4d992c0-9320-4632-907d-3fb68b39d3aa',
 '779183f7-98a6-448c-97df-323fc865d2d8',
 '33672e6c-62e5-46c9-aeee-a6ed4cf8aab6',
 '63cd36e4-5269-481d-8ec8-4e1d57dcc426',
 '87d3c28f-9b0a-411c-a1eb-ed905eef05bd',
 '361f6b21-9e68-40de-b524-851f9cc608ea']

In [16]:
# Data Retrieve (just for testing)
results = vector_store.similarity_search(
    "Designs and optimizes production processes",
    k=2,
    filter={"source": "website"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

    # Answer

* Process Engineer: Responsibilities: Designs and optimizes production processes to ensure efficiency and quality. Works closely with QA and QC teams to implement improvements. [{'source': 'website'}]
* Process Engineer: Responsibilities: Designs and optimizes production processes to ensure efficiency and quality. Works closely with QA and QC teams to implement improvements. [{'source': 'website'}]


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [12]:
def answer_to_user(query:str):

  # Vector Search

  vector_results =vector_store.similarity_search(query, k=2)

  print(len(vector_results))

  # pass to model vector results + user query

  final_answer = llm.invoke(f"ANSWER THIS USER QUERY{query}, Here are some reference to answer {vector_results}")

  return final_answer


In [17]:
# Pass the question to llm.
answer = answer_to_user("Designs and optimizes production processes")

2


In [18]:
# Here Answer to my question.
answer.content

'A Process Engineer designs and optimizes production processes to improve efficiency and quality.  This includes working closely with Quality Assurance (QA) and Quality Control (QC) teams to implement process improvements.'